In [45]:
import numpy as np   #importing libraries
import simpy 
import queue
import random
import random as rd
import matplotlib 
from matplotlib import pyplot as plt

In [46]:
global SZ,CP,RM
SZ=[]
CP=[]
RM=[]
N=1000
k=10
Lambda = 1
mu = 1.25
#t = 0   #system clock
t1=0
t2=0
t_lambda = 0 #poisson random variable used to generate next service time
t_mu = 0  #poisson random variable used to generate next departure time
t_a=0
wait=0
R=1000
transmission_time=0
l="0"
P_transmission=500
P_idle=100


In [47]:
class Packet(object):
    def __init__(self, time, id, mobileID, seeed):
        
        self.time=time                                                
        self.id=id
        self.mobileID=mobileID   
        self.size=random.randint(200,400) #seed 1 op 2, seed 2 op 
        SZ.append(round(self.size,2))
        #print(seeed)
        #random.seed(seeed)
        self.cpu=random.randint(20,90) #seed 1 op 2, seed 2 op 
        CP.append(round(self.cpu,2))
        #random.seed(seeed)
        self.ram=random.randint(1, 32) #seed 1 op 3, seed 2 op 1, seed 3 op 4
        RM.append(round(self.ram,2))

class mobile(object): #mobile class with an ID              
    def __init__(self, ID):
        self.ID=ID
        self.pot=int(1)
        
    #generates packets and sends via out_pipe
    def packet_generator(self, numPackets, env, out_pipe, task_id,send): 
        
        yield env.timeout(send)
        print("sending task {} of mobile {} at time {}".format(task_id, self.ID, env.now))
        p=Packet(env.now, task_id, self.ID, self.pot)
        self.pot+=1
        if(self.pot==4):
                self.pot=1
        yield out_pipe.put(p)
        item = yield out_pipe.get()



In [48]:
 def reward_generator(T): 
    Tsla=2.5 
    Nsla=3
    Gsla=3  
    Cost_sla=50
    Cg=0
    Ct=0
    Cn=0
    Cost_n=0
    
    #if (G<Gsla):
        #Cg=-5*(Gsla/G)
    if (T>Tsla):
        Ct = -5 * (T/Tsla)
    #if (N>Nsla):
        #Cn=-5*(N/Nsla)
    #if (cost>Cost_sla):
        #Cost_n=-5*(cost/Cost_sla)
    
    Reward = (Tsla/T) + Ct #Reward Function
    return Reward

In [49]:
class Fog(object):      
    def __init__(self, ID):
        self.ID=ID
        self.Number_queue=0
        self.lost=0
        self.response_time=0
        self.server_count=0
        
        self.service_time=[]
        self.response_time_list=[]
        self.Serverqueue=[]
        self.waitlist=[]
        self.energy_offloading=[]
        self.reward_list=[]
        
    def FOG(self, env, server, in_pipe, mobile):
        
         msg = yield in_pipe.get()
         transmission_time = (msg.size/R)
         #print("transmits task {} to need time {}".format(msg.id,transmission_time,self.ID))
         yield env.timeout(transmission_time)
         a=env.now
         print("Arrived task {} at time {} at FOG {} from mobile {}".format(msg.id,a,self.ID,mobile))
         if self.server_count==3:
            if len(self.Serverqueue)<k:
               self.Serverqueue.append(msg)
               #print(len(self.Serverqueue))
               self.Number_queue = self.Number_queue + 1
            
            else:
                self.lost=self.lost+1 
                return  
                 
         u=rd.random()
         t_mu=-np.log(u)/mu
         self.service_time.append(t_mu)
        
         E = (P_transmission * transmission_time) + (P_idle * t_mu)
         self.energy_offloading.append(E)
        
         with server.request() as req:
             yield req
             s=env.now
             wait=s-a
             #print("Starting service task {} at time {} at FOG {}".format(msg.id,s,self.ID))
             #print("Waiting time of task {} at time {} at FOG {}".format(msg.id,wait,self.ID))
             self.waitlist.append(wait)
             self.response_time=wait+t_mu
             self.response_time_list.append(self.response_time)
             
             #if wait>0:
              #  if len(self.Serverqueue)<2:
               #     self.Serverqueue.append(msg)
                #    self.Number_queue = self.Number_queue + 1
                 #   yield env.timeout(t_mu)
                    #print("Finish service task {} at time {} at FOG {}".format(msg.id,env.now,self.ID))
                  #  if len(self.Serverqueue)>0:
                   #     self.Serverqueue.pop(0)
                #else:
                   # self.lost=self.lost+1 
                   # return 
             self.server_count=self.server_count+1
             yield env.timeout(t_mu)
             #print("Finish service task {} at time {} at FOG {}".format(msg.id,env.now,self.ID))
             self.server_count=self.server_count-1            
             if len(self.Serverqueue)>0:
                 self.Serverqueue.pop(0)
             r=reward_generator(self.response_time)
             self.reward_list.append(r)

In [50]:
env1 = simpy.Environment()    
server = simpy.Resource(env1, capacity=3)

pipe1 = simpy.Store(env1)
pipe2 = simpy.Store(env1)
pipe3 = simpy.Store(env1)

ms1=mobile(1)
ms2=mobile(2)
ms3=mobile(3)

fog1 = Fog(1)
fog2 = Fog(2)
fog3=Fog(3)

for i in range(N):
    
    choose=random.randint(1,2)
    
    if(choose==1):
        u=rd.random()
        t_lambda=-np.log(u)/Lambda
        t_a=t1+t_lambda
        t1=t_a

        env1.process(ms1.packet_generator(1, env1, pipe1, i,t_a))
        env1.process(fog1.FOG(env1, server, pipe1,ms1.ID))

        u=rd.random()
        t_lambda=-np.log(u)/Lambda
        t_a=t2+t_lambda
        t2=t_a

        env1.process(ms2.packet_generator(1, env1, pipe2, i,t_a))
        env1.process(fog1.FOG(env1, server, pipe2,ms2.ID))
        
    if(choose==2):
        u=rd.random()
        t_lambda=-np.log(u)/Lambda
        t_a=t1+t_lambda
        t1=t_a

        env1.process(ms1.packet_generator(1, env1, pipe1, i,t_a))
        env1.process(fog2.FOG(env1, server, pipe1,ms1.ID))

        u=rd.random()
        t_lambda=-np.log(u)/Lambda
        t_a=t2+t_lambda
        t2=t_a

        env1.process(ms2.packet_generator(1, env1, pipe2, i,t_a))
        env1.process(fog2.FOG(env1, server, pipe2,ms2.ID))
    
env1.run()

print(fog1.lost,fog2.lost)
#print(fog1.waitlist)
#Na=N-fog.lost
#print("No    response_time       reward")
#i=0
#while (i<N):
 #   print("{}     {}     {}".format(i+1,fog1.response_time_list[i],fog1.reward_list[i]))
 #   i+=1
#print("FOG-1 MAXIMUM REAWARD:", max(fog1.reward_list))
#print("FOG-1 total number of task in queue",fog1.Number_queue)
avgwaittime=sum(fog1.waitlist)/N
print("FOG-1 Average queueing time:", avgwaittime)
avgservicetime=sum(fog1.service_time)/N
print("FOG-1 Average service time:", avgservicetime)
avgsystemtime=sum(fog1.response_time_list)/N
print("FOG-1 Average response time:", avgsystemtime)
avgenergy=sum(fog1.energy_offloading)/N
print("FOG-1 Average Energy consumption:", avgenergy)

#print("No    response_time       reward")
#i=0
#while (i<N):
 #   print("{}     {}     {}".format(i+1,fog2.response_time_list[i],fog2.reward_list[i]))
  #  i+=1
#print("FOG-2 MAXIMUM REAWARD:", max(fog2.reward_list))
#print("FOG-2 total number of task in queue",fog2.Number_queue)
avgwaittime=sum(fog2.waitlist)/N
print("FOG-2 Average queueing time:", avgwaittime)
avgservicetime=sum(fog2.service_time)/N
print("FOG-2 Average service time:", avgservicetime)
avgsystemtime=sum(fog2.response_time_list)/N
print("FOG-2 Average response time:", avgsystemtime)
avgenergy=sum(fog2.energy_offloading)/N
print("FOG-2 Average Energy consumption:", avgenergy)


sending task 0 of mobile 2 at time 1.8474030266233379
Arrived task 0 at time 2.0924030266233378 at FOG 2 from mobile 2
sending task 1 of mobile 2 at time 2.1407746632236417
Arrived task 1 at time 2.454774663223642 at FOG 1 from mobile 2
sending task 0 of mobile 1 at time 2.5180066601317566
Arrived task 0 at time 2.8190066601317567 at FOG 2 from mobile 1
sending task 1 of mobile 1 at time 3.4719118895274064
Arrived task 1 at time 3.803911889527406 at FOG 1 from mobile 1
sending task 2 of mobile 1 at time 4.161851948526864
Arrived task 2 at time 4.4778519485268635 at FOG 1 from mobile 1
sending task 2 of mobile 2 at time 4.502603461977879
Arrived task 2 at time 4.788603461977878 at FOG 1 from mobile 2
sending task 3 of mobile 2 at time 5.148647008413504
sending task 3 of mobile 1 at time 5.456380979455582
Arrived task 3 at time 5.504647008413504 at FOG 1 from mobile 2
Arrived task 3 at time 5.811380979455581 at FOG 1 from mobile 1
sending task 4 of mobile 2 at time 5.844275642642604
send